In [1]:
import BioSimSpace as bss

/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/jguven/Software/miniconda3/envs/sire/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)



Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



In [ ]:
solvent = "TIP3P"
box_edges = 20
box_type = "orthorhombic"

In [2]:
protein_path = "../inputs/protein/"
ligand_path = "../inputs/ligands/"
ligand_name_1 = "ligand_8_xtal.sdf"
ligand_name_2 = "docked_10.sdf"
ligand_1 = bss.IO.readMolecules(ligand_path+ligand_name_1)[0]
ligand_2 = bss.IO.readMolecules(ligand_path+ligand_name_2)[0]
ligand_1_params = bss.Parameters.gaff2(ligand_1, net_charge=-1).getMolecule()
ligand_2_params = bss.Parameters.gaff2(ligand_2, net_charge=-1).getMolecule()

In [ ]:
for ligand_params in [ligand_1_params, ligand_2_params]:
    